In [18]:
pip install datasets

In [19]:
import torch
from torch.autograd import Variable

import numpy as np

In [20]:
from datasets import load_dataset
dataset = load_dataset("liar")

Using custom data configuration default
Reusing dataset liar (/root/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514)


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
data_train = dataset["train"]
data_valid = dataset["validation"]
data_test = dataset["test"]

In [22]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [24]:
import re
NUM_SAMPLE = 10

bag_of_words = set()
# 1. Get Bag of Words (this example only does 10 samples from training set)
iter = 1
for sample_i in data_train:
  sample_i_text = re.sub(r'[^\w\s]', '', sample_i['statement'])
  sample_i_tokens = word_tokenize(sample_i_text)
  filtered_tokens = [tok.lower() for tok in sample_i_tokens if not tok.lower() in stop_words]
  bag_of_words |= set(filtered_tokens)

  # TODO: Delete the next 3 lines (just there for prototyping purposes)
  if iter == NUM_SAMPLE:
    break
  iter += 1

# 2. Map each word in BoW to an index
bow_map = {}
index = 0
for i in bag_of_words:
  bow_map[i] = index
  index += 1

# 3. Bag of Words encoding
x_data = torch.zeros(NUM_SAMPLE, len(bag_of_words))
y_data = torch.zeros(NUM_SAMPLE, 1)
iter = 1
for index, sample_i in enumerate(data_train):
  #print(sample_i)
  sample_i_text = re.sub(r'[^\w\s]', '', sample_i['statement'])
  sample_i_tokens = word_tokenize(sample_i_text)
  for tok in sample_i_tokens:
    if not tok.lower() in stop_words:
      x_data[index][bow_map[tok.lower()]] += 1
  
  if (sample_i['label'] > 0) and (sample_i['label'] < 4):
    y_data[index] = 1

  # TODO: Delete the next 3 lines (just there for prototyping purposes)
  if iter == NUM_SAMPLE:
    break
  iter += 1

#print(x_data)
#print(y_data)

In [25]:
class LogisticRegression(torch.nn.Module):
     def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(len(bag_of_words), 1)
     def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LogisticRegression()
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [26]:
x_input = Variable(x_data)
y_input = Variable(y_data)

In [27]:
for epoch in range(20):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(x_data)
    # Compute Loss
    loss = criterion(y_pred, y_data)
    # Backward pass
    loss.backward()
    optimizer.step()

In [50]:
# Just for a sample test (using sample index 23 in the training dataset)
new_x = torch.zeros(1, len(bag_of_words))
new_x_text = re.sub(r'[^\w\s]', '', data_train[23]['statement'])
new_x_tokens = word_tokenize(new_x_text)
for tok in new_x_tokens:
  if (not tok.lower() in stop_words) and (tok.lower() in bow_map):
    new_x[0][bow_map[tok.lower()]] += 1
print(new_x)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])


In [51]:
y_pred = model(new_x)
print(y_pred)

tensor([[0.4961]], grad_fn=<SigmoidBackward0>)


In [52]:
print("predicted Y value: ", y_pred.data[0][0])

predicted Y value:  tensor(0.4961)
